In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pd.set_option('display.max_columns', None)
df=pd.read_csv("../input/customer-personality-analysis/marketing_campaign.csv",sep="\t")
df.head()

In [3]:
df.info()

In [4]:
df.describe().T

In [5]:
df_coulmns = list(df.columns)
df_coulmns

In [6]:
for i in df_coulmns:
    print(f"category {i} has :", df[i].nunique(), "unique values.")

In [7]:
df['Age'] = 2014 - df['Year_Birth']
df.head()

In [8]:
df = df.drop("Year_Birth", axis=1)

In [9]:
df['Education'].value_counts()

In [10]:
df['Marital_Status'].value_counts()

In [11]:
dict1 = {'Graduation':'UG', 'Master':'PG', '2n Cycle':'UG', 'PhD': 'PG', 'Basic':'UG'}

In [12]:
dict2= {'Married':'Together', 'Divorced':'Single','Widow': 'Single','Alone':'Single', 'Absurd':'Single','YOLO':'Single'}

In [13]:
df['Marital_Status']= df['Marital_Status'].replace(dict2)

In [14]:
df['Education'] = df['Education'].replace(dict1)

In [15]:
df.isnull().sum()

In [16]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.NaN,
                        strategy ='mean')
                    
imputer = imputer.fit(df[['Income']])
df['Income'] = imputer.transform(df[['Income']])
df

In [17]:
df.isnull().sum()

In [18]:
df['Childern_home'] = df['Kidhome']+df['Teenhome']
df = df.drop(['Kidhome','Teenhome'], axis=1)
df.shape

In [19]:
df.head(5)

In [20]:
df['Childern_home'].value_counts()

In [21]:
df["Dt_Customer"] = pd.to_datetime(df["Dt_Customer"], dayfirst=True)

In [22]:
df["Total_spend"] = df["MntWines"]+ df["MntFruits"]+ df["MntMeatProducts"]+ df["MntFishProducts"]+ df["MntSweetProducts"]+ df["MntGoldProds"]

In [23]:
df = df.rename(columns={"MntWines": "Wines","MntFruits":"Fruits","MntMeatProducts":"Meat","MntFishProducts":"Fish","MntSweetProducts":"Sweets","MntGoldProds":"Gold"})
df

In [24]:
datelist = []
for i in df["Dt_Customer"]:
    datelist.append(i.date())

duration_list = []
for i in df["Dt_Customer"]:
    duration_list.append(max(datelist) - i.date())
print(max(datelist))   
df["dayscustomerfor"] = duration_list
df['dayscustomerfor']

In [25]:
df["dayscustomerfor"] = df["dayscustomerfor"].apply(lambda x: int(str(x).split(" ")[0]))
df.head()

In [26]:
# we create masks with quantile method to filter out data and categorise them
masknew = df["dayscustomerfor"] <= df["dayscustomerfor"].quantile(0.3)
maskold = df["dayscustomerfor"] > df["dayscustomerfor"].quantile(0.3)
df.loc[masknew, "Cust_Seniority"] = "New"
df.loc[maskold, "Cust_Seniority"] = "Old"
df["Cust_Seniority"].value_counts() # we display the New and Old value counts

In [27]:
dict3 = {0:'No Child', 1: 'One Child', 2:'two_Children', 3:'3_Children'}

In [28]:
df["Total_campaigns_accepted"] = df['AcceptedCmp1'] + df['AcceptedCmp2'] + df['AcceptedCmp3'] + df['AcceptedCmp4'] + df['AcceptedCmp5'] + df["Response"]

In [29]:
df["Couple"] = df["Marital_Status"].apply(lambda x: 1 if x == "Together"  else 0)

In [30]:
to_drop = ['ID','Dt_Customer', 'Recency', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Response', 'dayscustomerfor']
df.drop(to_drop, axis=1, inplace=True)

In [31]:
df.head()

In [32]:
df.shape

In [33]:
df.groupby("Education")["Total_spend"].describe().round(2)


In [34]:
df.groupby("Education")["Income"].describe().round(2)

In [35]:
import seaborn as sns
import matplotlib.pyplot as plt

In [36]:
plt.figure(figsize=(10,5))
sns.histplot(data=df, x='Income')

In [37]:
plt.figure(figsize=(10,5))
sns.histplot(data=df, x='Age')

In [38]:
df = df[df["Income"] <=160000]
df = df[df["Age"] < 100]

In [39]:
plt.figure(figsize=(10,5))
sns.histplot(data=df, x='Age')

In [40]:
plt.figure(figsize=(10,5))
sns.histplot(data=df, x='Income')

In [41]:
df.groupby("Education")["Income"].describe().round(2)

In [42]:
df.groupby("Total_campaigns_accepted")["Income"].describe().round(2)


In [43]:
from sklearn.preprocessing import LabelEncoder

In [44]:
label_encoder =LabelEncoder()
# Encode labels in column. 
df['Education']= label_encoder.fit_transform(df['Education'])
df.head(10)

In [45]:
df.drop('Marital_Status', axis=1, inplace=True)

In [46]:
df["Cust_Seniority"] = df["Cust_Seniority"].replace({"Old":0, "New":1})

In [47]:
from sklearn.preprocessing import StandardScaler

In [48]:
# scaled our data using standard scaler
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_scaled.head()

In [49]:
df.drop(['Z_CostContact', 'Z_Revenue'], axis=1, inplace=True)

In [50]:
plt.figure(figsize=(20,20))
sns.heatmap(df.corr(), annot=True, fmt=".2f", cmap="Blues")
plt.show()

In [51]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

In [52]:
to_keep = ['Education', 'Income', 'Cust_Seniority', 'Age',
       'Couple', 'Childern_home', 'Total_spend']

elbow = KElbowVisualizer(KMeans(), k=8)
elbow.fit(df[to_keep])
elbow.show()
plt.show()

In [53]:
# we fit and predict the clusters and add the cluster numbers back to our dataframe
kmeancluster = KMeans(n_clusters=4)
df["kcluster"] = kmeancluster.fit_predict(df[to_keep])

In [54]:
# we check the cluster counts
sns.countplot(data=df, x="kcluster", palette="tab20")
plt.title("Cluster Distribution")
plt.xlabel("Cluster")
plt.show()


In [55]:
pl = sns.scatterplot(data = df,x=df["Total_spend"], y=df["Income"],hue=df["kcluster"], palette="tab20")
pl.set_title("Income And Spending w/ Cluster")
plt.legend()
plt.show()

In [56]:
# we check the descriptive stats of income for each clusters
df.groupby("kcluster")["Income"].describe().round(2)

In [57]:
df.groupby("kcluster")["Total_spend"].describe().round(2)

In [59]:
sns.boxenplot(x=df["kcluster"], y=df["Income"], palette="tab20")
plt.show()

In [60]:
sns.boxenplot(x=df["kcluster"], y=df["Total_spend"], palette="tab20")
plt.show()

In [61]:
df.groupby("kcluster")[['Wines', 'Fruits', 'Meat', 'Fish', 'Sweets', 'Gold']].mean().round(2)

In [62]:
df["Age_bins"] = pd.cut(df["Age"], bins=[20,35,50,65,80])
sns.countplot(x=df["kcluster"], hue=df["Age_bins"], palette="tab20")
plt.show()

In [63]:
sns.countplot(data=df, x="kcluster", hue="Education", palette="tab20")
plt.legend(title='Education', labels=['UG','PG'])
plt.show()

In [64]:
sns.countplot(data=df, x="kcluster", hue="Childern_home", palette="tab20")
plt.show()

In [65]:
# we plot boxen plot
sns.boxenplot(x=df["kcluster"], y=df["Total_campaigns_accepted"], palette="tab20")
plt.show()

In [66]:
# we plot a countplot
sns.countplot(data=df, x="kcluster", hue="Cust_Seniority", palette="tab20")
plt.legend(title='Old/New Customer', labels=['Old', 'New'])
plt.show()

In [67]:
sns.countplot(data=df, x="kcluster", hue="Couple", palette="tab20")
plt.legend(title='Couple', labels=['No', 'Yes'], loc="upper center")
plt.show()

In [68]:
sns.boxenplot(x=df["kcluster"], y=df["NumWebVisitsMonth"], palette="tab20")
plt.show()

In [69]:
df.groupby("kcluster")['NumWebPurchases'].describe().round(2)

In [70]:
sns.boxenplot(x=df["kcluster"], y=df["NumWebPurchases"], palette="tab20")
plt.show()

In [71]:
import plotly.graph_objects as go

In [73]:
PLOT = go.Figure()
for C in list(df.kcluster.unique()):
    

    PLOT.add_trace(go.Scatter3d(x = df[df.kcluster == C]['Income'],
                                y = df[df.kcluster == C]['Total_campaigns_accepted'],
                                z = df[df.kcluster == C]['Total_spend'],                        
                                mode = 'markers',marker_size = 6, marker_line_width = 1,
                                name = str(C)))
PLOT.update_traces(hovertemplate='Income: %{x} <br>Seniority: %{y} <br>Spending: %{z}')

    
PLOT.update_layout(width = 800, height = 800, autosize = True, showlegend = True,
                   scene = dict(xaxis=dict(title = 'Income', titlefont_color = 'black'),
                                yaxis=dict(title = 'Total_campaigns_accepted', titlefont_color = 'black'),
                                zaxis=dict(title = 'Spending', titlefont_color = 'black')),
                   font = dict(family = "Gilroy", color  = 'black', size = 12))